In [65]:
from sklearn.model_selection import train_test_split
import json
import numpy as np
import pandas as pd
from nltk.stem import PorterStemmer
from stop_words import get_stop_words
import unicodedata
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import random
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [16]:
data = {}
with open('C:/Users/noeli/Desktop/Practica_NLP/Video_Games_5.json/Video_Games_5.json', "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        data[idx] = json.loads(line)

In [25]:
def sentence_normalization(sentence):
    sentence = unicodedata.normalize('NFKD', sentence).lower().encode('ascii', errors='ignore').decode('utf-8')
    sentence = re.sub(' +', ' ', ' '.join([word if word.isalpha() else '' for word in sentence.split()])).strip()
    return sentence


def remove_stopwords(sentence, sw_list):
    sentence = ' '.join([word for word in sentence.split() if word not in sw_list])
    return sentence


stemmer = PorterStemmer()
def apply_stemming(sentence): 
    return ' '.join([stemmer.stem(word) for word in sentence.split()])

sw_list = get_stop_words('en')

def process_reviews(reviews, sw_list):
    processed_sentences = []
    for sent in reviews:
        if not sent != sent:
            sent = sentence_normalization(sent)
            sent = remove_stopwords(sent, sw_list)
            sent = apply_stemming(sent)
            processed_sentences.append(sent)
        else:
            processed_sentences.append('None')
    return processed_sentences

In [30]:
processed_reviews = process_reviews(df_reviews['review'], sw_list)

In [31]:
df_reviews['processedReview'] = processed_reviews
df_reviews['processedReview'] = processed_reviews 
df_reviews['processedReview'] = df_reviews['processedReview'].replace('', np.nan) 
df_reviews = df_reviews.dropna(subset=['processedReview']) 

In [34]:
def label_sentiment(row): 
    if int(row['sentiment']) < 3:
        return 1 # NEGATIVA
    else:
        return 0 # POSITIVA

In [35]:
df_reviews = df_reviews.copy() 
df_reviews['sentiment_label'] = df_reviews.apply(lambda row: label_sentiment(row), axis=1) 

In [43]:
# Dividimos los datos en entrenamiento (75%) y prueba (25%) para entrenar y evaluar el modelo de clasificación

X_train, X_test, y_train, y_test = train_test_split(
    df_reviews['processedReview'],
    df_reviews['sentiment_label'],
    train_size=0.75,
    test_size=0.25,
    random_state=42,
    shuffle=True
)

In [ ]:
# Creamos y ajustamos un vectorizador TF-IDF para transformar el texto en vectores numéricos
# Se limita a palabras que aparecen en al menos 3 documentos y en no más del 95% del corpus

cv = TfidfVectorizer(
    max_df=0.95, # con este parámetro elimino los términos que son demasiado comunes y así reduzco el ruido
    min_df=3,    #
    max_features=2500,
    strip_accents='ascii',
    ngram_range=(1, 1)
)

cv.fit(X_train)

TfidfVectorizer(max_df=0.95, max_features=2500, min_df=3, strip_accents='ascii')

In [45]:
# Mostramos las primeras 20 palabras del vocabulario generado por el vectorizador y el tamaño total del vocabulario

print(list(cv.vocabulary_.items())[:20])
print(len(cv.vocabulary_))

[('get', 915), ('visual', 2387), ('beauti', 188), ('game', 893), ('fit', 821), ('song', 2044), ('dog', 602), ('end', 687), ('never', 1449), ('hesit', 1020), ('buy', 284), ('made', 1292), ('releas', 1780), ('date', 514), ('know', 1188), ('will', 2442), ('worth', 2466), ('price', 1657), ('even', 715), ('half', 981)]
2500


In [42]:
# Aplicamos la transformación TF-IDF a los datos de entrenamiento y prueba para convertir el texto en vectores numéricos

Xtrain = cv.transform(X_train)
Xtest = cv.transform(X_test)

In [49]:
# Obtenemos las 10 palabras más comunes en el conjunto de entrenamiento antes de vectorizar

all_words = ' '.join(X_train).split()
wordsexample = [word for word, _  in Counter(all_words).most_common(10)]

In [50]:
# Muestra el valor IDF de cada palabra en 'wordsexample', indicando 'OOV' si la palabra no está en el vocabulario

vocab = dict(zip(cv.get_feature_names_out(), cv.idf_))

print('{0:20}{1:20}'.format('Palabra', 'IDF'))
for word in wordsexample:
    if word not in vocab:
        print('{0:20}{1:20}'.format(word, 'OOV'))
    else:
        print('{0:20}{1:2.3f}'.format(word, vocab[word]))

Palabra             IDF                 
game                1.604
play                2.079
like                2.196
can                 2.423
get                 2.308
just                2.375
one                 2.368
will                2.666
great               2.412
realli              2.675


In [53]:
# Selecciona y muestra una reseña aleatoria del conjunto de entrenamiento junto con su sentimiento

i = random.randint(0, len(X_train))
print('ID: {}'.format(i))
print('Sentiment: {}'.format(y_train.iloc[i]))
print('Review: {}'.format(X_train.iloc[i]))

ID: 279850
Sentiment: 0
Review: love lego game one great game actual beat


In [54]:
# Extrae y muestra las 10 palabras con mayor y menor TF-IDF en la reseña seleccionada

doc_vector = Xtrain[i]
df_tfidf = pd.DataFrame(doc_vector.T.todense(), index=cv.get_feature_names_out(), columns=['tfidf'])
df_tfidf = df_tfidf[df_tfidf['tfidf'] > 0]

top_n = 10
print('Top {} palabras con mayor TF-IDF en la reseña {}:\n{}'.format(top_n, i, df_tfidf.sort_values(by=["tfidf"],ascending=False)[:top_n]))
print('\nTop {} palabras con menor TF-IDF en la reseña {}:\n{}'.format(top_n, i, df_tfidf.sort_values(by=["tfidf"],ascending=False)[-top_n:]))

Top 10 palabras con mayor TF-IDF en la reseña 279850:
           tfidf
lego    0.629834
beat    0.421850
actual  0.359368
game    0.323215
love    0.275169
great   0.243060
one     0.238625

Top 10 palabras con menor TF-IDF en la reseña 279850:
           tfidf
lego    0.629834
beat    0.421850
actual  0.359368
game    0.323215
love    0.275169
great   0.243060
one     0.238625


In [59]:
# Entrena un modelo de regresión logística para distintos valores de C y evalúa su precisión en entrenamiento y test

c_params = [0.01, 0.05, 0.25, 0.5, 1, 10, 100, 1000, 10000]

train_acc = list()
test_acc = list()
for c in c_params:
    lr = LogisticRegression(C=c, solver='lbfgs', max_iter=500)
    lr.fit(Xtrain, y_train)

    train_predict = lr.predict(Xtrain)
    test_predict = lr.predict(Xtest)

    print ("Accuracy for C={}: {}".format(c, accuracy_score(y_test, test_predict)))

    train_acc.append(accuracy_score(y_train, train_predict))
    test_acc.append(accuracy_score(y_test, test_predict))

Accuracy for C=0.01: 0.891663261136085
Accuracy for C=0.05: 0.9085328974254189
Accuracy for C=0.25: 0.9159460563955865
Accuracy for C=0.5: 0.9165100122599101
Accuracy for C=1: 0.916616264814058
Accuracy for C=10: 0.9170412750306498
Accuracy for C=100: 0.9169023293829178
Accuracy for C=1000: 0.9170167552104618
Accuracy for C=10000: 0.9169431957498978


In [ ]:
# Cogemos el parámetro con la precisión más alta, en este caso es C = 10 y volvemos a entrenar el modelo de LogisticRegression con él

lr = LogisticRegression(C=10, solver='lbfgs', max_iter=500)
lr.fit(Xtrain, y_train)

test_predictlr = lr.predict(Xtest)

In [72]:
# Muestra la matriz de confusión, el reporte de clasificación y la precisión del modelo en el conjunto de test

print('Matriz de confusión:\n{}'.format(confusion_matrix(y_test, test_predictlr)))
print('\nReporte de clasificación:\n{}'.format(classification_report(y_test, test_predictlr)))
print('Precisión del modelo:{}'.format(accuracy_score(y_test, test_predictlr)))

Matriz de confusión:
[[106392   2296]
 [  7854   5808]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.93      0.98      0.95    108688
           1       0.72      0.43      0.53     13662

    accuracy                           0.92    122350
   macro avg       0.82      0.70      0.74    122350
weighted avg       0.91      0.92      0.91    122350

Precisión del modelo:0.9170412750306498


In [70]:
# Entrena un clasificador Naive Bayes multinomial, predice en test y muestra métricas de evaluación

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

nb = MultinomialNB()
nb.fit(Xtrain, y_train)

test_pred = nb.predict(Xtest)

print("\n=== Test Evaluation ===")
print("Precisión:", accuracy_score(y_test, test_pred))
print(confusion_matrix(y_test, test_pred))
print(classification_report(y_test, test_pred))


=== Test Evaluation ===
Precisión: 0.8944993870044953
[[108595     93]
 [ 12815    847]]
              precision    recall  f1-score   support

           0       0.89      1.00      0.94    108688
           1       0.90      0.06      0.12     13662

    accuracy                           0.89    122350
   macro avg       0.90      0.53      0.53    122350
weighted avg       0.90      0.89      0.85    122350



ELECCIÓN DE MODELO

El primer modelo (LogisticRegression) tiene mejor precisión global, es significativamente mejor en recall y f1-score. El modelo de Naive Bayes multinomial casi no detecta la clase 1 ya que el recall de 0.06 es bastante bajo. Además tiene una precisión altísima en clase 0, pero falla mucho en detectar la clase 1.
En cuanto a Macro F1 y Weighted F1 también domina bastante el modelo de LogisticRegression, nos encontramos valores más altos.
Tras estudiar los resultados de cada modelo decido elegir el modelo LogisticRegression ya que tiene mayor precisión, tiene mejor rendimiento en la clase minoritaria (en este caso la clase 1), el F1 es más equilibrado y el recall es mayor en la clase 1 con lo cual detecta más casos positivos reales.